In [1]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET # Biblioteca nativa para ler XML
import json
import time


URL_API = "https://alegre-es.portaltp.com.br/api/transparencia.asmx/json_pagamentos"

dados_acumulados = []

print("--- INÍCIO DA EXTRAÇÃO ---")

# Loop 1: Anos
for ano in range(2015, 2026):
    # Loop 2: Meses (Janeiro a Dezembro)
    for mes in range(1, 13):
        print(f"Baixando {mes:02d}/{ano}...", end="")

        # Parâmetros que a API espera (Query String)
        params = {"ano": ano, "mes": mes}

        try:
            # 1. Bate na porta do servidor (GET)
            response = requests.get(URL_API, params=params, timeout=10)

            if response.status_code == 200:
                # 2. Quebra a casca do XML
                # A API devolve: <string> ...conteudo json... </string>
                # O comando .fromstring converte o texto da resposta em objeto XML
                xml_root = ET.fromstring(response.content)

                # O .text pega o conteúdo de dentro da tag <string>
                conteudo_json = xml_root.text

                # 3. Lê o JSON (se existir conteúdo)
                if conteudo_json:
                    dados = json.loads(conteudo_json) # Converte string JSON para Lista Python

                    if len(dados) > 0:
                        # Transforma lista de dicionários em DataFrame e guarda
                        df_temp = pd.DataFrame(dados)
                        dados_acumulados.append(df_temp)
                        print(f" OK! ({len(dados)} registros)")
                    else:
                        print(" Vazio.")
                else:
                    print(" XML vazio.")
            else:
                print(f" Erro HTTP: {response.status_code}")

        except Exception as e:
            print(f" Falha de conexão: {e}")

        # Pequena pausa para não ser bloqueado pelo servidor (Good Citizen)
        # time.sleep(0.1)

# 4. Consolidação
if dados_acumulados:
    # 'concat' cola todos os pedacinhos (dataframes mensais) em um só
    df_final = pd.concat(dados_acumulados, ignore_index=True)

    # Salvo em CSV
    df_final.to_csv("dados_brutos.csv", index=False, encoding='utf-8-sig')
    print(f"\nSucesso! Arquivo 'dados_brutos.csv' salvo com {len(df_final)} linhas.")
else:
    print("Nenhum dado foi baixado.")

--- INÍCIO DA EXTRAÇÃO ---
Baixando 01/2015... OK! (777 registros)
Baixando 02/2015... OK! (623 registros)
Baixando 03/2015... OK! (788 registros)
Baixando 04/2015... OK! (749 registros)
Baixando 05/2015... OK! (772 registros)
Baixando 06/2015... OK! (894 registros)
Baixando 07/2015... OK! (782 registros)
Baixando 08/2015... OK! (851 registros)
Baixando 09/2015... OK! (699 registros)
Baixando 10/2015... OK! (1023 registros)
Baixando 11/2015... OK! (804 registros)
Baixando 12/2015... OK! (999 registros)
Baixando 01/2016... OK! (727 registros)
Baixando 02/2016... OK! (584 registros)
Baixando 03/2016... OK! (887 registros)
Baixando 04/2016... OK! (762 registros)
Baixando 05/2016... OK! (862 registros)
Baixando 06/2016... OK! (969 registros)
Baixando 07/2016... OK! (848 registros)
Baixando 08/2016... OK! (861 registros)
Baixando 09/2016... OK! (751 registros)
Baixando 10/2016... OK! (882 registros)
Baixando 11/2016... OK! (844 registros)
Baixando 12/2016... OK! (1201 registros)
Baixando 01

In [2]:
df = pd.read_csv('/content/dados_brutos.csv')

In [9]:
import pandas as pd
import requests
URL_SIDRA = 'https://apisidra.ibge.gov.br/values/t/6579/n6/3200201/v/all/p/all'

print(f'------ INICIANDO EXTRAÇÃO DOS DADOS ------: {URL_SIDRA}')

try:
  response = requests.get(URL_SIDRA, timeout=15)

  if response.status_code == 200:
    data = response.json()

    df_ibge_raw = pd.DataFrame(data)
  else:
    print(f"Erro na api: {response.status_code}")
except Exception as e:
  print(f"Falha na conexão: {e}")
  exit()


------ INICIANDO EXTRAÇÃO DOS DADOS ------: https://apisidra.ibge.gov.br/values/t/6579/n6/3200201/v/all/p/all


In [10]:
df_ibge_raw.head()

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Município (Código),Município,Variável (Código),Variável,Ano (Código),Ano
1,6,Município,45,Pessoas,31878,3200201,Alegre (ES),9324,População residente estimada,2001,2001
2,6,Município,45,Pessoas,31986,3200201,Alegre (ES),9324,População residente estimada,2002,2002
3,6,Município,45,Pessoas,32112,3200201,Alegre (ES),9324,População residente estimada,2003,2003
4,6,Município,45,Pessoas,32377,3200201,Alegre (ES),9324,População residente estimada,2004,2004


In [33]:
dados_brutos_ibge = "dados_brutos_ibge.csv"
df_ibge_raw.to_csv(dados_brutos_ibge, index=False)

In [12]:
df_ibge = df_ibge_raw[['V','D3N']]

In [13]:
df_ibge.head()

,V,D3N
0,Valor,Ano
1,31878,2001
2,31986,2002
3,32112,2003
4,32377,2004


In [17]:
df_ibge = df_ibge.rename(columns={'V': 'populacao_ano', 'D3N': 'ano'})

In [22]:
df_ibge = df_ibge[df_ibge['ano'].str.isnumeric()]

In [25]:
df_ibge['ano'] = df_ibge['ano'].astype(int)
df_ibge['populacao_ano'] = pd.to_numeric(df_ibge['populacao_ano'], errors='coerce')

In [26]:
df_ibge.head()

,populacao_ano,ano
1,31878,2001
2,31986,2002
3,32112,2003
4,32377,2004
5,32523,2005


In [28]:
df_ibge_final = df_ibge[(df_ibge['ano'] >= 2015) & (df_ibge['ano'] <= 2025)]

In [29]:
df_ibge_final

,populacao_ano,ano
13,32205,2015
14,32175,2016
15,32146,2017
16,30568,2018
17,30084,2019
18,29975,2020
19,29869,2021
20,30744,2024
21,30702,2025


In [32]:
dados_final_ibge = "dados_final_ibge.csv"
df_ibge_final.to_csv(dados_final_ibge, index=False)

In [3]:
df.head()

,ano,mes,unidade_gestora,data,especie,empenho,liquidacao,pagamento,tipo_pagamento,elemento_despesa,subtitulo,funcao,subfuncao,programa,fonte_recurso,grupo_despesa,documento_favorecido,nome_favorecido,valor
0,2015,01 - Janeiro,Fundo de Educacao de Alegre,2015-01-13T00:00:00,Original,NaN,NaN,0000023/2015,Extra Orcamentario,218810102001 - INSS SERVICOS DE TERCEIROS PF/PJ,NaN,NaN,NaN,NaN,1103000000 - FUNDEB - PAGAMENTO DOS PROFISSION...,NaN,29.979.036/0001-40,INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS,132.00
1,2015,01 - Janeiro,Prefeitura Municipal de Alegre,2015-01-28T00:00:00,Original,NaN,NaN,0000178/2015,Extra Orcamentario,113810600005 - FGTS EDUCACAO,NaN,NaN,NaN,NaN,1000000000 - RECURSOS ORDINARIOS,NaN,00.360.305/0001-04,CAIXA ECONOMICA FEDERAL,152.02
2,2015,01 - Janeiro,Prefeitura Municipal de Alegre,2015-01-28T00:00:00,Original,NaN,NaN,0000179/2015,Extra Orcamentario,113810600006 - FGTS ACAO SOCIAL,NaN,NaN,NaN,NaN,1000000000 - RECURSOS ORDINARIOS,NaN,00.360.305/0001-04,CAIXA ECONOMICA FEDERAL,2669.82
3,2015,01 - Janeiro,Prefeitura Municipal de Alegre,2015-01-28T00:00:00,Original,NaN,NaN,0000198/2015,Extra Orcamentario,113810600011 - RPPS SOCIAL EMPRESA,NaN,NaN,NaN,NaN,1000000000 - RECURSOS ORDINARIOS,NaN,05.269.863/0001-82,INSTITUTO DE PREVIDENCIA E ASSISTENCIA DO MUNI...,2912.42
4,2015,01 - Janeiro,Prefeitura Municipal de Alegre,2015-01-28T00:00:00,Original,NaN,NaN,0000199/2015,Extra Orcamentario,113810600012 - RPPS SOCIAL ALIQUOTA SUPLEMENTAR,NaN,NaN,NaN,NaN,1000000000 - RECURSOS ORDINARIOS,NaN,05.269.863/0001-82,INSTITUTO DE PREVIDENCIA E ASSISTENCIA DO MUNI...,3904.64


In [4]:
df.describe()

,ano,valor
count,118170.000000,1.181700e+05
mean,2020.206575,8.998155e+03
std,3.161403,3.360458e+04
min,2015.000000,-7.250000e+05
25%,2018.000000,2.579300e+02
50%,2020.000000,1.266500e+03
75%,2023.000000,5.063488e+03
max,2025.000000,2.974786e+06


In [5]:
df.columns

Index(['ano', 'mes', 'unidade_gestora', 'data', 'especie', 'empenho',
       'liquidacao', 'pagamento', 'tipo_pagamento', 'elemento_despesa',
       'subtitulo', 'funcao', 'subfuncao', 'programa', 'fonte_recurso',
       'grupo_despesa', 'documento_favorecido', 'nome_favorecido', 'valor'],
      dtype='object')

In [7]:
df['funcao']

,funcao
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
118165,10 - SAUDE
118166,15 - URBANISMO
118167,10 - SAUDE
118168,15 - URBANISMO


In [36]:
df1 = df[['funcao', 'unidade_gestora']]

In [37]:
df1.head()

,funcao,unidade_gestora
0,NaN,Fundo de Educacao de Alegre
1,NaN,Prefeitura Municipal de Alegre
2,NaN,Prefeitura Municipal de Alegre
3,NaN,Prefeitura Municipal de Alegre
4,NaN,Prefeitura Municipal de Alegre


In [44]:
df['unidade_gestora'].isna().sum()

np.int64(0)

In [47]:
print(len(df))

118170


In [46]:
print(df['unidade_gestora'].value_counts())

unidade_gestora
Prefeitura Municipal de Alegre                               39507
Fundo de Saude de Alegre                                     35650
Fundo de Educacao de Alegre                                  22487
Fundo de Assistencia Social de Alegre                        15655
SAAE SERVICO AUTONOMO DE AGUA E ESGOTO                        4171
FAFIA  FACULDADE DE FILOSOFIA CIENCIAS E LETRAS DE ALEGRE      700
Name: count, dtype: int64


In [28]:
qtd_nulos = df1.isna().sum()
print(qtd_nulos)

22646


In [29]:
# Mostra quantas vezes cada categoria aparece + os NaN
print(df1.value_counts(dropna=False))

funcao
10 - SAUDE                   29482
NaN                          22646
04 - ADMINISTRACAO           19604
12 - EDUCACAO                15703
08 - ASSISTENCIA SOCIAL      11407
15 - URBANISMO                5614
20 - AGRICULTURA              3241
28 - ENCARGOS ESPECIAIS       3156
18 - GESTAO AMBIENTAL         2374
17 - SANEAMENTO               1531
13 - CULTURA                  1323
02 - JUDICIARIA                828
25 - ENERGIA                   567
27 - DESPORTO E LAZER          350
06 - SEGURANCA PUBLICA         195
23 - COMERCIO E SERVICOS       145
16 - HABITACAO                   2
19 - CIENCIA E TECNOLOGIA        2
Name: count, dtype: int64


In [33]:
total = len(df1)
nulos = df1.isna().sum()
porc = (nulos/total) * 100
print(porc)

19.163916391639162
